# Tabular data example

In [199]:
from velour import Dataset, Model, Datum, Annotation, GroundTruth, Prediction, Label
from velour.enums import TaskType
from velour import schemas

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [200]:
from velour.client import Client, ClientException

client = Client("http://localhost:8000")

Succesfully connected to http://localhost:8000/.


In [201]:
dset = load_breast_cancer()
X, y, target_names = dset["data"], dset["target"], dset["target_names"]
X_train, X_test, y_train, y_test = train_test_split(X, y)

pipe = make_pipeline(StandardScaler(), LogisticRegression())

In [202]:
X_train.shape, y_train[:4], target_names

((426, 30), array([0, 1, 1, 1]), array(['malignant', 'benign'], dtype='<U9'))

In [203]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [204]:
y_train_probs = pipe.predict_proba(X_train)
y_test_probs = pipe.predict_proba(X_test)

In [205]:
y_train_probs[:4]

array([[9.99747105e-01, 2.52895124e-04],
       [6.39394501e-06, 9.99993606e-01],
       [9.82825958e-06, 9.99990172e-01],
       [1.09285419e-01, 8.90714581e-01]])

## Velour Dataset ingestion

We now ingest the groundtruth labels into velour. For each sample, velour expects a list of `Label` objects. Each `Label` has a key and value. Allowing key/value labels and having a single row be annotated by multiple labels supports multi-label classification.

In this example there's just a single label per element and we'll set the class key to "class". The `add_groundtruth` method returns the ids of the newly created groundtruth.

Create a Velour `Dataset` for the train and test sets respectively.

In [206]:
# create train dataset
velour_train_dataset = Dataset(client, "breast-cancer-train", reset=True)

# create test dataset
velour_test_dataset = Dataset(client, "breast-cancer-test", reset=True)

Create groundtruths using Velour's `GroundTruth` object.

In [207]:
# format training groundtruths
training_groundtruths = [
    GroundTruth(
        datum=Datum(
            uid=f"train{i}",
        ),
        annotations=[
            Annotation(
                task_type=TaskType.CLASSIFICATION,
                labels=[Label(key="class", value=target_names[t])]
            )
        ]
    )
    for i, t in enumerate(y_train)
]

# format testing groundtruths
testing_groundtruths = [
    GroundTruth(
        datum=Datum(
            uid=f"test{i}",
        ),
        annotations=[
            Annotation(
                task_type=TaskType.CLASSIFICATION,
                labels=[Label(key="class", value=target_names[t])]
            )
        ]
    )
    for i, t in enumerate(y_test)
]

Use `Dataset.add_groundtruth` for data ingestion to the backend. 

In [208]:
# add the training groundtruths
for gt in training_groundtruths:
    velour_train_dataset.add_groundtruth(gt)

# add the testing groundtruths
for gt in testing_groundtruths:
    velour_test_dataset.add_groundtruth(gt)

Finalize datasets, necessary for evaluation

In [209]:
velour_train_dataset.finalize()
velour_test_dataset.finalize()

<Response [200]>

## Model inference ingestion

Now we create a velour model and post the predictions on the two datasets. Each prediction should be a list of `ScoredLabel`, which consist of a label and a confidence score. The confidence scores over all of the classes in a key must sum to (approximately) 1.

Create Velour `Model`

In [210]:
velour_model = Model(client, "breast-cancer-linear-model", reset=True)

Format predictions

In [211]:
training_predictions = [
    Prediction(
        datum=Datum(
            dataset=velour_train_dataset.name,
            uid=f"train{i}",
        ),
        annotations=[
            Annotation(
                task_type=TaskType.CLASSIFICATION,
                labels=[
                    Label(
                        key="class", 
                        value=target_names[j],
                        score=p,
                    )                        
                    for j, p in enumerate(prob)
                ]
            )
        ]
    )
    for i, prob in enumerate(y_train_probs)
]

testing_predictions = [
    Prediction(
        datum=Datum(
            dataset=velour_test_dataset.name,
            uid=f"test{i}",
        ),
        annotations=[
            Annotation(
                task_type=TaskType.CLASSIFICATION,
                labels=[
                    Label(
                        key="class",
                        value=target_names[j],
                        score=p,
                    )                        
                    for j, p in enumerate(prob)
                ]
            )
        ]
    )
    for i, prob in enumerate(y_test_probs)
]

In [212]:
# add the train predictions
for pd in training_predictions:
    velour_model.add_prediction(pd)

# add the test predictions
for pd in testing_predictions:
    velour_model.add_prediction(pd)

finalize models, necessary for evaluation

In [213]:
velour_model.finalize_inferences(velour_train_dataset)
velour_model.finalize_inferences(velour_test_dataset)

evaluate

In [214]:
train_eval_job = velour_model.evaluate_classification(velour_train_dataset)
train_eval_job.wait_for_completion()
results = train_eval_job.results()

In [215]:
results.dataframe

value
dataset                                           breast-cancer-train
type      parameters             label                               
Accuracy  {"label_key": "class"} n/a                         0.990610
F1        "n/a"                  class: benign               0.992453
                                 class: malignant            0.987578
Precision "n/a"                  class: benign               0.988722
                                 class: malignant            0.993750
ROCAUC    {"label_key": "class"} n/a                         0.998784
Recall    "n/a"                  class: benign               0.996212
                                 class: malignant            0.981481

In [216]:
results.confusion_matrices

[{'label_key': 'class',
  'entries': [{'prediction': 'benign', 'groundtruth': 'benign', 'count': 263},
   {'prediction': 'benign', 'groundtruth': 'malignant', 'count': 3},
   {'prediction': 'malignant', 'groundtruth': 'benign', 'count': 1},
   {'prediction': 'malignant', 'groundtruth': 'malignant', 'count': 159}]}]

In [217]:
test_eval_job = velour_model.evaluate_classification(velour_test_dataset)
test_eval_job.wait_for_completion()
results = train_eval_job.results()

In [218]:
results.dataframe

value
dataset                                           breast-cancer-train
type      parameters             label                               
Accuracy  {"label_key": "class"} n/a                         0.990610
F1        "n/a"                  class: benign               0.992453
                                 class: malignant            0.987578
Precision "n/a"                  class: benign               0.988722
                                 class: malignant            0.993750
ROCAUC    {"label_key": "class"} n/a                         0.998784
Recall    "n/a"                  class: benign               0.996212
                                 class: malignant            0.981481

In [219]:
results.confusion_matrices

[{'label_key': 'class',
  'entries': [{'prediction': 'benign', 'groundtruth': 'benign', 'count': 263},
   {'prediction': 'benign', 'groundtruth': 'malignant', 'count': 3},
   {'prediction': 'malignant', 'groundtruth': 'benign', 'count': 1},
   {'prediction': 'malignant', 'groundtruth': 'malignant', 'count': 159}]}]

## Sanity check scikit-learn classification report

In [220]:
from sklearn.metrics import classification_report

In [221]:
y_train_preds = pipe.predict(X_train)

In [222]:
print(classification_report(y_train, y_train_preds, digits=6, target_names=target_names))

              precision    recall  f1-score   support

   malignant   0.993750  0.981481  0.987578       162
      benign   0.988722  0.996212  0.992453       264

    accuracy                       0.990610       426
   macro avg   0.991236  0.988847  0.990015       426
weighted avg   0.990634  0.990610  0.990599       426

